In [1]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

In [2]:
sc.version

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1716570764976_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'3.0.1-amzn-0'

In [3]:
sc.install_pypi_package("pandas==1.0.5", "https://pypi.org/simple")
sc.install_pypi_package("scipy==1.4.1", "https://pypi.org/simple")
sc.install_pypi_package("matplotlib==3.2.1", "https://pypi.org/simple")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#sc.install_pypi_package("setuptools", "https://pypi.org/simple")
#sc.install_pypi_package("pyarrow==10.0.0", "https://pypi.org/simple")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import hour
#import pyarrow.parquet as pq

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
nyc_taxi_rides_2019 = spark.read.csv('s3://nyc-tlc/csv_backup/yellow_tripdata_2019*.csv', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Read in New York EV data from s3 bucket
#ny_ev_parquet = spark.read.parquet('s3://final-project-nrel-stations-jy/station-parquet-files/NY_stations.parquet', header=True)

#ny_ev_df = pd.read_parquet('s3://final-project-nrel-stations-jy/station-parquet-files/NY_stations.parquet', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
nyc_taxi_rides_2019.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)

In [7]:
nyc_taxi_rides_2019.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

84399019

In [8]:
nyc_taxi_rides_2019.createOrReplaceTempView("taxi")

query1 = '''
        SELECT DOLocationID, AVG(trip_distance) as avg_distance
        FROM taxi
        GROUP BY DOLocationID
        ORDER BY DOLocationID
        '''

nyc_taxi_rides_2019_query1 = spark.sql(query1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
nyc_taxi_rides_2019_query1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------------+
|DOLocationID|      avg_distance|
+------------+------------------+
|           1|17.360809109244844|
|          10| 5.289382681650723|
|         100|2.1536543639538386|
|         101|12.604187705034942|
|         102| 8.544133596837945|
|         105|3.4645977011494256|
|         106| 7.034566023166024|
|         107| 2.045252362732258|
|         108|13.540840189873423|
|         109|22.639713438735175|
|          11|13.694749090909092|
|         110|18.637999999999998|
|         111| 9.102529252925294|
|         112| 5.846527929206396|
|         113|2.0858439844625853|
|         114|2.1229795831612104|
|         115|17.801793842034805|
|         116| 5.032623748609566|
|         117|13.163150303030303|
|         118|20.073691087260848|
+------------+------------------+
only showing top 20 rows

In [35]:
nyc_taxi_rides_2019.createOrReplaceTempView("taxi")

query2 = '''
        SELECT PULocationID, AVG(passenger_count) as PU_avg_passenger_count, 
            AVG(trip_distance) as PU_avg_distance, 
            AVG(RatecodeID) as PU_avg_ratecodeid,
            AVG(fare_amount) as PU_avg_fare_amount,
            AVG(total_amount) as PUavg_total_amount,
            COUNT(*) as PU_trip_counts
        FROM taxi
        GROUP BY PULocationID
        ORDER BY PULocationID
        '''

nyc_taxi_rides_2019_query2 = spark.sql(query2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
nyc_taxi_rides_2019_query2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------------+------------------+------------------+------------------+------------------+--------------+
|PULocationID|PU_avg_passenger_count|   PU_avg_distance| PU_avg_ratecodeid|PU_avg_fare_amount|PUavg_total_amount|PU_trip_counts|
+------------+----------------------+------------------+------------------+------------------+------------------+--------------+
|           1|    1.5099015634047481|1.5862343822304485|4.6961204400694845| 80.79781698287827| 96.73014576584902|          8644|
|          10|    1.5457224141434667| 13.43787625686659| 2.184312131680553| 48.44953548138391| 62.33970284943277|         31129|
|         100|    1.5467372471745167|2.3588217856962768|1.0369056446999052|11.781947835220183| 17.19327392334489|       1725187|
|         101|    1.1862275449101796|  8.50551184622262|1.9610778443113772| 34.43057666517659| 37.71014751899865|          2237|
|         102|     1.365021328458257| 5.369618502622796|1.5776965265082268|25.272136385312344|28.

In [37]:
nyc_taxi_rides_2019.createOrReplaceTempView("taxi")

query3 = '''
        SELECT DOLocationID, AVG(passenger_count) as DO_avg_passenger_count, 
            AVG(trip_distance) as DO_avg_distance, 
            AVG(RatecodeID) as DO_avg_ratecodeid,
            AVG(fare_amount) as DO_avg_fare_amount,
            AVG(total_amount) as DO_avg_total_amount,
            COUNT(*) as DO_trip_counts
        FROM taxi
        GROUP BY DOLocationID
        ORDER BY DOLocationID
        '''

nyc_taxi_rides_2019_query3 = spark.sql(query3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
nyc_taxi_rides_2019_query3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------------+------------------+------------------+------------------+-------------------+--------------+
|DOLocationID|DO_avg_passenger_count|   DO_avg_distance| DO_avg_ratecodeid|DO_avg_fare_amount|DO_avg_total_amount|DO_trip_counts|
+------------+----------------------+------------------+------------------+------------------+-------------------+--------------+
|           1|     1.637576089186786| 17.36080910924484|3.1567152269566603| 71.86014944676779|  98.74907813621495|        175514|
|          10|     1.525942539446875| 5.289382681650724|1.1481054443795113| 18.66474985294574| 21.696166217763842|         64602|
|         100|    1.5851915249524664|2.1536543639538386|1.0424801602617668|11.796396349918988| 17.356972831796732|       1335532|
|         101|    1.4816307549217176|12.604187705034944|1.3946674934118741|39.259592069604906|   44.6067907573813|          7011|
|         102|    1.5389315498307323| 8.544133596837945|1.0894228387416398|29.020188932806

In [40]:
# Coalesce to a single partition
single_part_df2 = nyc_taxi_rides_2019_query2.coalesce(1)
single_part_df3 = nyc_taxi_rides_2019_query3.coalesce(1)

# Write the dataframe to a single csv and a single parquet file that can be downloaded from S3
single_part_df2.write.parquet("s3://elec-transit-y-jy/nyc_taxi_rides_2019_aggByPU.parquet", mode="overwrite")
single_part_df3.write.parquet("s3://elec-transit-y-jy/nyc_taxi_rides_2019_aggByDO.parquet", mode="overwrite")
single_part_df2.write.csv("s3://elec-transit-y-jy/nyc_taxi_rides_2019_aggByPU.csv", header=True, mode="overwrite")
single_part_df3.write.csv("s3://elec-transit-y-jy/nyc_taxi_rides_2019_aggByDO.csv", header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
nyc_taxi_rides_2019.createOrReplaceTempView("taxi")

query4 = '''
        SELECT 
            hour(tpep_pickup_datetime) AS hour_of_day,
            PULocationID,
            COUNT(*) AS pickup_count
        FROM 
            taxi
        GROUP BY 
            hour_of_day,
            PULocationID
        ORDER BY
            hour_of_day,
            PULocationID
        '''

nyc_taxi_rides_2019_query4 = spark.sql(query4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
nyc_taxi_rides_2019_query4.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------+------------+
|hour_of_day|PULocationID|pickup_count|
+-----------+------------+------------+
|          0|           1|          28|
|          0|          10|         621|
|          0|         100|       49034|
|          0|         101|          26|
|          0|         102|          30|
|          0|         105|           4|
|          0|         106|         670|
|          0|         107|       64333|
|          0|         108|          20|
|          0|         109|           2|
|          0|          11|          14|
|          0|         111|           5|
|          0|         112|        2671|
|          0|         113|       37910|
|          0|         114|       79286|
|          0|         115|           7|
|          0|         116|        3801|
|          0|         117|          34|
|          0|         118|           9|
|          0|         119|         116|
+-----------+------------+------------+
only showing top 20 rows

In [27]:
# Coalesce to a single partition
single_part_df4 = nyc_taxi_rides_2019_query4.coalesce(1)

# Write the dataframe to a single csv file that can be downloaded from S3
single_part_df4.write.csv("s3://elec-transit-y-jy/nyc_taxi_rides_2019_aggByPUandHour_single.csv", header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
nyc_taxi_rides_2019.createOrReplaceTempView("taxi")

query5 = '''
        SELECT 
            hour(tpep_pickup_datetime) AS hour_of_day,
            DOLocationID,
            COUNT(*) AS dropoff_count
        FROM
            taxi
        GROUP BY 
            hour_of_day,
            DOLocationID
        ORDER BY
            hour_of_day,
            DOLocationID
        '''

nyc_taxi_rides_2019_query5 = spark.sql(query5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
nyc_taxi_rides_2019_query5.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------+-------------+
|hour_of_day|DOLocationID|dropoff_count|
+-----------+------------+-------------+
|          0|           1|          262|
|          0|          10|         2948|
|          0|         100|        30752|
|          0|         101|          251|
|          0|         102|          798|
|          0|         105|            2|
|          0|         106|         2670|
|          0|         107|        61470|
|          0|         108|          262|
|          0|         109|           86|
|          0|          11|          330|
|          0|         111|           71|
|          0|         112|        13395|
|          0|         113|        28661|
|          0|         114|        31902|
|          0|         115|           99|
|          0|         116|        16761|
|          0|         117|          349|
|          0|         118|          131|
|          0|         119|         1194|
+-----------+------------+-------------+
only showing top

In [21]:
nyc_taxi_rides_2019_query5.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6275

In [28]:
# Coalesce to a single partition
single_part_df5 = nyc_taxi_rides_2019_query5.coalesce(1)

# Write the dataframe to a single csv file that can be downloaded from S3
single_part_df5.write.csv("s3://elec-transit-y-jy/nyc_taxi_rides_2019_aggByDOandHour_single.csv", header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
nyc_taxi_rides_2019.createOrReplaceTempView("taxi")

query6 = '''
        SELECT 
            DOLocationID,
            COUNT(*) AS dropoff_count,
            flag_dropoff_peak_hours
        FROM (
            SELECT 
                hour(tpep_dropoff_datetime) AS dropoff_hour_of_day,
                DOLocationID,
                CASE 
                    WHEN hour(tpep_dropoff_datetime) BETWEEN 8 AND 23 THEN 1
                    ELSE 0
                END AS flag_dropoff_peak_hours
            FROM 
                taxi
        ) subquery
        GROUP BY 
            DOLocationID,
            flag_dropoff_peak_hours
        ORDER BY
            DOLocationID,
            flag_dropoff_peak_hours
        '''

nyc_taxi_rides_2019_query6 = spark.sql(query6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
nyc_taxi_rides_2019_query6.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+-----------------------+
|DOLocationID|dropoff_count|flag_dropoff_peak_hours|
+------------+-------------+-----------------------+
|           1|        33316|                      0|
|           1|       142198|                      1|
|          10|        10740|                      0|
|          10|        53862|                      1|
|         100|       167250|                      0|
|         100|      1168282|                      1|
|         101|         1079|                      0|
|         101|         5932|                      1|
|         102|         3460|                      0|
|         102|         9190|                      1|
|         105|           11|                      0|
|         105|           76|                      1|
|         106|         9798|                      0|
|         106|        29052|                      1|
|         107|       273558|                      0|
|         107|      1462356|                  

In [19]:
# Coalesce to a single partition
single_part_df6 = nyc_taxi_rides_2019_query6.coalesce(1)

# Write the dataframe to a single csv file that can be downloaded from S3
single_part_df6.write.csv("s3://elec-transit-y-jy/nyc_taxi_rides_2019_flagForPeakHours.csv", header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…